Revoir minimum WR distances : très long 

In [262]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from utils import simulate_tournament, simulate_n_tournaments, compare_settings, initiate_league


New LDC format places of interest :
* The 8 first teams qualify directly to round of 16
* From 9 to 24 : pseudo round of 32
* Interesting teams : i (9th) and y(25th)  

Format :
* 36 teams,
* 8 games,
* Hat determination of the first 2 games

LoL :
* 16 teams,
* Top 8 to Quarter finales,
* 6 games (not exactly, teams with 3 win qualifies, team with 3 loss gets eliminated)


In [263]:
from utils import assign_opponents, play_round
n_teams = 8
n_games = 4


In [293]:
simulate_tournament(10,5, delta_level = 'exponential',verbose = True)


Successfull allocation based on ranking
Round number 1, game g vs b of levels 0.02 vs 0.5: victory for b 
Round number 1, game e vs d of levels 0.06 vs 0.12: victory for e - UNEXPECTED 
Round number 1, game i vs a of levels 0.0 vs 1.0: victory for a 
Round number 1, game j vs f of levels 0.0 vs 0.03: victory for f 
Round number 1, game h vs c of levels 0.01 vs 0.25: victory for c 
Successfull allocation based on ranking
Round number 2, game b vs e of levels 0.5 vs 0.06: victory for b 
Round number 2, game a vs f of levels 1.0 vs 0.03: victory for a 
Round number 2, game c vs g of levels 0.25 vs 0.02: victory for c 
Round number 2, game d vs i of levels 0.12 vs 0.0: victory for d 
Round number 2, game j vs h of levels 0.0 vs 0.01: victory for h 
Successfull allocation based on ranking
Round number 3, game b vs a of levels 0.5 vs 1.0: victory for b - UNEXPECTED 
Round number 3, game c vs e of levels 0.25 vs 0.06: victory for c 
Round number 3, game f vs d of levels 0.03 vs 0.12: victory 

,Level,Strategy,Nb_win,Nb_games,Win_rate,R1_opponent,R1_result,R2_opponent,R2_result,R3_opponent,R3_result,R4_opponent,R4_result,R5_opponent,R5_result
b,0.5,[],5,5,1.0,g,Win,e,Win,a,Win,c,Win,i,Win
c,0.25,[],4,5,0.8,h,Win,g,Win,e,Win,b,Loss,a,Win
f,0.03125,[],3,5,0.6,j,Win,a,Loss,d,Loss,e,Win,g,Win
d,0.125,[],3,5,0.6,e,Loss,i,Win,f,Win,a,Loss,h,Win
a,1.0,[],3,5,0.6,i,Win,f,Win,b,Loss,d,Win,c,Loss
g,0.015625,[],2,5,0.4,b,Loss,c,Loss,h,Win,j,Win,f,Loss
h,0.007812,[],2,5,0.4,c,Loss,j,Win,g,Loss,i,Win,d,Loss
e,0.0625,[],2,5,0.4,d,Win,b,Loss,c,Loss,f,Loss,j,Win
i,0.003906,[],1,5,0.2,a,Loss,d,Loss,j,Win,h,Loss,b,Loss
j,0.001953,[],0,5,0.0,f,Loss,h,Loss,i,Loss,g,Loss,e,Loss


In [289]:
compare_settings(n_tournaments=1000, 
                 n_teams=16, 
                 n_rounds = 6, 
                 delta_level ='linear',
                 strategies={'i':[1]}, 
                 probabilistic=True, 
                 deterministic=True)


Probabilistic resolution


KeyboardInterrupt: 